# Project 1 - Finite differences

Many physics codes only provide objective values and are not able to compute their gradients. In that case, the derivative is often approximated using finite differences. The goal of this project is to implement your own finite difference approximation and investigate the impact of round-off error as well as the increase in runtime compared to analytical gradient computation.

In [ ]:
# Lets begin by setting up our toy problem again.
from pyplasmaopt import *
nfp = 2
(coils, currents, magnetic_axis, eta_bar) = get_24_coil_data(nfp=nfp, ppp=8, at_optimum=False)
stellarator = CoilCollection(coils, currents, nfp, True)
iota_target = 0.103
coil_length_target = 4.398229715025710
magnetic_axis_length_target = 6.356206812106860
eta_bar = -2.25
obj = SimpleNearAxisQuasiSymmetryObjective(
        stellarator, magnetic_axis, iota_target, eta_bar=eta_bar,
        coil_length_target=coil_length_target, magnetic_axis_length_target=magnetic_axis_length_target)

A first order approximation to the derivative at $x$ in direction of the $i$-th degree of freedom is given by
$$ (\nabla f(x))_i= \frac{f(x+\epsilon e_i) - f(x)}{\epsilon} + O(\epsilon). $$

The following code will compute the gradient in a direction $h$ and compares it to the derivative that the code computes for us. Note that we pass `compute_derivative=False` to the `update` function. That's a bit quicker than just calling update, as it will skip those computations that are not needed for the objective value.

In [ ]:
def compute_directional_derivative_fd(obj, x, h, eps):
    obj.update(x, compute_derivative=False)
    fx = obj.res
    obj.update(x+eps*h, compute_derivative=False)
    fxh = obj.res
    return (fxh-fx)/eps

x = obj.x0
h = np.random.rand(*(x.shape))
obj.update(x)
derivative = np.sum(obj.dres*h)
eps = 1e-6
fd_derivative = compute_directional_derivative_fd(obj, x, h, eps)
print('Exact derivative =', derivative)
print('Finite differences derivative =', fd_derivative)
print('Error =', abs(derivative-fd_derivative))

## Tasks

1. Vary the stepsize $\epsilon$. As $\epsilon\to0$, how does the error behave?
2. Look up the 2nd, 4th, and 6th order finite difference stencils on wikipedia and implement them. Show a plot comparing the step size $\epsilon$ with the approximation error. What do you observe?
3. The `minimize` function in scipy supports finite differences to compute the gradient. To use this, set the argument `jac=False` and make sure that the function you pass to `minimize` only returns the objective value, and not both objective and gradient. Now run the optimisation once with exact gradients and once using the scipy finite difference approximation for 500 iterations. Compare the runtime of the two and plot the objective values. What to do you observe? Call `print` on the object that `minimize` returns to find out why each algorithm terminated.

*Note*: If you pass `callback=obj.callback` to `minimize`, then the objective history will be stored in `obj.Jvals`. Beore running a second optimisation, you can reset this history by calling `obj.clear_history()`.

## Solutions